## Access Token 

In [22]:
url = "https://api.trendkite.com/api/login"

payload="{\"username\":\"murali.ravikumar@externals.adidas-group.com\",\"password\":\"passwordforcision@123\"}"
headers = {
  'username': 'murali.ravikumar@externals.adidas-group.com',
  'password': 'passwordforcision@123',
  'Content-Type': 'text/plain',

}

response = requests.request("POST", url, headers=headers, data=payload)
response.text

'{"username":"murali.ravikumar@externals.adidas-group.com","roles":["ROLE_ALERTS","ROLE_DASHBOARD","ROLE_NEWS","ROLE_REPORTS","ROLE_USER"],"access_token":"a2d4b8rvssc7bd9vi9333avs806hr2kb"}'

## Mentions of BLOG/Article on different platforms

In [ ]:
import requests

auth_token = 'e4tnkqrdji2gnalt74f5vrfo0fgcauh0'
headers = {
  'X-Auth-Token': auth_token ,
}

searchid='3607569'
range_start = '2020-01-01T12:00:56.235-0700'
range_end = '2020-11-06T12:59:56.235-0700'

url = "https://api.trendkite.com/api/v2/totalmentions?s="+searchid+"&range-start="+range_start+"&range-end="+range_end+"&page-size=500&sort=asc"

response = requests.get(url, headers=headers)

response.json()

## Stats

In [37]:
%%time
from datetime import datetime, timedelta
import re
import pandas as pd
import time
import requests

def dayLevelAggre():
    auth_token = 'pene081mcojhcekl5a8gst0q6tno3hs0'
    start_date = datetime.strptime('2020-08-01','%Y-%m-%d')
    end_date = datetime.strptime('2020-11-09','%Y-%m-%d')
    # calculating difference between startdate and enddate
    date_range = end_date -start_date
    date_range = int(str(date_range).split()[0])

    range_start = '' 
    range_end =''
    searchid='3607569'

    Aggregated_stats = []
    #creating day wise data from start and end date
    for i in range(date_range+1):
        #adding the days to start date
        range_start = start_date + timedelta(days=i)
        range_start = re.sub(' ','T',datetime.strftime(range_start,'%Y-%m-%d 00:00:00'))+".000Z"
        range_end =re.sub('00:00:00.000Z','23:59:59.000Z',range_start)
        stats=requests.get('http://api.trendkite.com/api/v2/stats?s='+searchid+'&range-start='+range_start+'&range-end='+range_end,
                           headers={'X-Auth-Token': auth_token})
        
        Aggregated_stats.append(stats.json())
        print('Appending entries', i)

    return Aggregated_stats

def aggregatedData():
    
    Aggregated_stats = dayLevelAggre()
    
    df_aggr = pd.DataFrame(columns=['Tool','Domain','searchId','searchName','startDate','endDate','ave_Newspaper',
                                   'ave_Blog','ave_Television','totalAVE','readership_Newspaper',
                                   'readership_Blog','readership_Television','totalReadership','totalMentions',
                                   'socialFacebookComments','socialFacebookLike','socialFacebookShare',
                                    'socialFacebookTotal','socialGoogleplus',
                                    'socialLinkedin','socialPinterest','socialReddit',
                                    'socialSum','socialTwitter',
                                   'totalSocialShares'])
    
    print('Creating dataset..')
    
    num = 0
    for k in Aggregated_stats:
        agger_data = k

        for i in range(len(agger_data)):
            df_aggr.loc[num]=''
            df_aggr['Tool'][num] = 'Cision'
            df_aggr['Domain'][num] = 'PR'
            df_aggr['searchId'][num] = agger_data[i]['searchId']
            df_aggr['searchName'][num] = agger_data[i]['searchName']
            df_aggr['startDate'][num] = agger_data[i]['startDate']
            df_aggr['endDate'][num] = agger_data[i]['endDate']

            temp_data = agger_data[i]['data']
            for j in range(len(temp_data)):
                if('ave' in temp_data[j].keys()):
                    df_aggr['ave_Newspaper'][num] = temp_data[j]['ave'][0]['value']
                    df_aggr['ave_Blog'][num] = temp_data[j]['ave'][1]['value']
                    df_aggr['ave_Television'][num] = temp_data[j]['ave'][2]['value']
                    df_aggr['totalAVE'][num] = temp_data[j]['totalAVE']
                if('readership' in temp_data[j].keys()):
                    df_aggr['readership_Newspaper'][num] = temp_data[j]['readership'][0]['value']
                    df_aggr['readership_Blog'][num] = temp_data[j]['readership'][1]['value']
                    df_aggr['readership_Television'][num] = temp_data[j]['readership'][2]['value']
                    df_aggr['totalReadership'][num] = temp_data[j]['totalReadership']
                if('totalMentions' in temp_data[j].keys()):
                    df_aggr['totalMentions'][num] = temp_data[j]['totalMentions']
                if('socialShares' in temp_data[j].keys()):
                    for key,value in temp_data[j]['socialShares'].items():
                        df_aggr[key][num] = temp_data[j]['socialShares'][key]
                    df_aggr['totalSocialShares'][num] = temp_data[j]['totalSocialShares']

        num+=1  
            
    return df_aggr

df = aggregatedData()
# df.to_csv('Aggregated.csv',index = False)

Appending entries 0
Appending entries 1
Appending entries 2
Appending entries 3
Appending entries 4
Appending entries 5
Appending entries 6
Appending entries 7
Appending entries 8
Appending entries 9
Appending entries 10
Appending entries 11


ConnectionError: HTTPSConnectionPool(host='api.trendkite.com', port=443): Max retries exceeded with url: /api/v2/stats?s=3607569&range-start=2020-08-13T00:00:00.000Z&range-end=2020-08-13T23:59:59.000Z (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B3EB413A60>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [21]:
df

Tool Domain searchId                             searchName  \
0    Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
1    Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
2    Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
3    Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
4    Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
..      ...    ...      ...                                    ...   
96   Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
97   Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
98   Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
99   Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   
100  Cision     PR  3607569  2020 - Campaign #ReadyforSport adidas   

                    startDate                   endDate ave_Newspaper  \
0    2020-08-01T00:00:00.000Z  2020-08-01T23:59:59.000Z        131915   
1    2020-08-02T00:00:00.000Z  2020-08-02T23:59:59.000Z           759   
2    2020-08-03T00:00:00.000Z  2020-08-03T23:59:59.000Z           156   
3    2020-08-04T00:00:00.000Z  2020-08-04T23:59:59.000Z        669146   
4    2020-08-05T00:00:00.000Z  2020-08-05T23:59:59.000Z          1281   
..                        ...                       ...           ...   
96   2020-11-05T00:00:00.000Z  2020-11-05T23:59:59.000Z           401   
97   2020-11-06T00:00:00.000Z  2020-11-06T23:59:59.000Z           475   
98   2020-11-07T00:00:00.000Z  2020-11-07T23:59:59.000Z             0   
99   2020-11-08T00:00:00.000Z  2020-11-08T23:59:59.000Z             0   
100  2020-11-09T00:00:00.000Z  2020-11-09T23:59:59.000Z             0   

    ave_Blog ave_Television totalAVE  ... socialFacebookLike  \
0          0              0   131915  ...                  0   
1         54              0      813  ...                  0   
2        960              0     1116  ...                  0   
3        744              0   669890  ...                  0   
4        168              0     1449  ...                  0   
..       ...            ...      ...  ...                ...   
96         0              0      401  ...                  0   
97         0            435      910  ...                  0   
98         0             61       61  ...                  0   
99         0              0        0  ...                  0   
100        0              0        0  ...                  0   

    socialFacebookShare socialFacebookTotal socialGoogleplus socialLinkedin  \
0                     0                  25                0              0   
1                     0                   0                0              0   
2                     0                   0                0              0   
3                     0                   0                0              0   
4                     0                   0                0              0   
..                  ...                 ...              ...            ...   
96                    0                  43                0              0   
97                    0                   0                0              0   
98                    0                   0                0              0   
99                    0                   0                0              0   
100                   0                   0                0              0   

    socialPinterest socialReddit socialSum socialTwitter totalSocialShares  
0                 0            0        27             2                27  
1                 0            0         4             4                 4  
2                 0            0         1             1                 1  
3                 0            0        17            17                17  
4                 0            0         0             0                 0  
..              ...          ...       ...           ...               ...  
96                0            0

In [ ]:
%%time
start_date = datetime.strptime('2020-08-01','%Y-%m-%d')
end_date = datetime.strptime('2020-08-31','%Y-%m-%d')

date_range = end_date -start_date
date_range = int(str(date_range).split()[0])

range_start = '' 
range_end =''
searchid='3607569'

Aggregated_stats = []
#creating day wise data from start and end date
for i in range(date_range+1):
    range_start = start_date + timedelta(days=i)
    range_start = re.sub(' ','T',datetime.strftime(range_start,'%Y-%m-%d 00:00:00'))+".000Z"
    range_end =re.sub('00:00:00.000Z','23:59:59.000Z',range_start)

    stats=requests.get('http://api.trendkite.com/api/v2/stats?s='+searchid+'&range-start='+range_start+'&range-end='+range_end,
                       headers={'X-Auth-Token': auth_token})

    Aggregated_stats.append(stats.json())

In [ ]:
Aggregated_stats

In [ ]:
from datetime import datetime
start_date = datetime.strptime('2020-08-01','%Y-%m-%d')
end_date = datetime.strptime('2020-08-31','%Y-%m-%d')

date_range = end_date -start_date
date_range = int(str(date_range).split()[0])

range_start = '' 
range_end =''
for i in range(date_range+1):
    range_start = start_date + timedelta(days=i)
    

stats=requests.get('http://api.trendkite.com/api/v2/stats?s=3607569&range-start=2020-08-01T00:00:00.000Z&range-end=2020-08-01T23:59:59.000Z',headers={'X-Auth-Token': auth_token})
#gives you aggregated data but didnt breakdown day wise or monthly
agger_data = stats.json()

In [ ]:


num = 0
for i in range(len(agger_data)):
    df_aggr.loc[num]=''
    df_aggr['Tool'][num] = 'Cision'
    df_aggr['Domain'][num] = 'PR'
    df_aggr['searchId'][num] = agger_data[i]['searchId']
    df_aggr['searchName'][num] = agger_data[i]['searchName']
    df_aggr['startDate'][num] = agger_data[i]['startDate']
    df_aggr['endDate'][num] = agger_data[i]['endDate']
    
    temp_data = agger_data[i]['data']
    for j in range(len(temp_data)):
        if('ave' in temp_data[j].keys()):
            df_aggr['ave_Newspaper'][num] = temp_data[j]['ave'][0]['value']
            df_aggr['ave_Blog'][num] = temp_data[j]['ave'][1]['value']
            df_aggr['ave_Television'][num] = temp_data[j]['ave'][2]['value']
            df_aggr['totalAVE'][num] = temp_data[j]['totalAVE']
        if('readership' in temp_data[j].keys()):
            df_aggr['readership_Newspaper'][num] = temp_data[j]['readership'][0]['value']
            df_aggr['readership_Blog'][num] = temp_data[j]['readership'][1]['value']
            df_aggr['readership_Television'][num] = temp_data[j]['readership'][2]['value']
            df_aggr['totalReadership'][num] = temp_data[j]['totalReadership']
        if('totalMentions' in temp_data[j].keys()):
            df_aggr['totalMentions'][num] = temp_data[j]['totalMentions']
        if('socialShares' in temp_data[j].keys()):
            for key,value in temp_data[j]['socialShares'].items():
                df_aggr[key][num] = temp_data[j]['socialShares'][key]
            df_aggr['totalSocialShares'][num] = temp_data[j]['totalSocialShares']
        
    num+=1    


In [ ]:
df_aggr

## SearchID

In [35]:
import requests

#method to generate access_token, hardcoded the values for username and password
def accessToken():
    
    access_token = ''
    url = "https://api.trendkite.com/api/login"

    payload="{\"username\":\"murali.ravikumar@externals.adidas-group.com\",\"password\":\"passwordforcision@123\"}"
    headers = {
      'username': 'murali.ravikumar@externals.adidas-group.com',
      'password': 'passwordforcision@123',
      'Content-Type': 'text/plain',

    }

    response = requests.request("POST", url, headers=headers, data=payload)
    res = response.text
    #response was in string in dictionary format so eval will convert that string to dictionary
    access_token = eval(res)['access_token']
    
    return access_token

def searchidData():
    auth_token = accessToken()
    df = pd.DataFrame(columns=['id','title','taxonomy'])
    
    headers = {
      'X-Auth-Token': auth_token ,
    }

    url = "https://api.trendkite.com/api/v2/searches"

    response = requests.get(url, headers=headers)

    data = response.json()
    datalist = data['searches']
    #creating dataset of id and title from json response
    num = 0
    for i in datalist:
        df.loc[num]=''
        for key,value in i.items():
            df[key][num] = i[key]
            
        num +=1
    
    df.to_csv('Search_ID_Dataset.csv',index = False)
        
        

In [36]:
searchidData()

In [28]:
headers = {
      'X-Auth-Token':'kd3ea84dboput5jeqpib8787d2c3ouj2',
    }

url = "https://api.trendkite.com/api/v2/searches"

response = requests.get(url, headers=headers)

response.json()

{'searches': [{'id': 4056738,
   'title': '2020 - Inflight - Y',
   'taxonomy': 'Custom'},
  {'id': 4063793, 'title': '2020 - Mutator Pack - Y', 'taxonomy': 'Custom'},
  {'id': 4063792, 'title': '2020 - Mutator Pack- Y', 'taxonomy': 'Custom'},
  {'id': 4063780, 'title': '2020 Hardwired)', 'taxonomy': 'Custom'},
  {'id': 4063737, 'title': '2020 Inflight Pack', 'taxonomy': 'Custom'},
  {'id': 4044411,
   'title': '2020 - adidas football - jersey analysis',
   'taxonomy': 'Custom'},
  {'id': 4048640,
   'title': '2019 - adidas Football - JERSEY',
   'taxonomy': 'Custom'},
  {'id': 4048642,
   'title': '2019 - adidas football - jersey HOME',
   'taxonomy': 'Custom'},
  {'id': 4044412,
   'title': '2020 - adidas football - jersey analysis',
   'taxonomy': 'Custom'},
  {'id': 3967249, 'title': 'US Open 2020', 'taxonomy': 'Custom'},
  {'id': 3967243, 'title': 'Prague Marathon', 'taxonomy': 'Custom'},
  {'id': 3947985, 'title': 'Dani - Japan Federation', 'taxonomy': 'Custom'},
  {'id': 3947937

In [26]:
url = "https://api.trendkite.com/api/login"

payload="{\"username\":\"murali.ravikumar@externals.adidas-group.com\",\"password\":\"passwordforcision@123\"}"
headers = {
  'username': 'murali.ravikumar@externals.adidas-group.com',
  'password': 'passwordforcision@123',
  'Content-Type': 'text/plain',

}

response = requests.request("POST", url, headers=headers, data=payload)
res = response.text
#response was in string in dictionary format so eval will convert that string to dictionary
access_token = eval(res)['access_token']

In [27]:
access_token

'kd3ea84dboput5jeqpib8787d2c3ouj2'